In [1]:
import pandas as pd
import numpy as np
import os
import glob

## 1. Renaming files with it's activity:

In [ ]:
rootdir = "D:/Ajit/pip/sample_labeled"
folder_list = []
for subdir, dirs, files in os.walk(rootdir):
    folder_list.append(subdir)
del folder_list[0]
folder_list

In [ ]:
for folder in folder_list:
    path=folder
    i=0
    ne=0
    ur=0
    df=0
    for filename in os.listdir(path):
    #     print(path+'/'+filename)
        x=path+'/'+filename
    #     print(x)
        if i%2!=0:
            tag=''
            df_json=pd.read_json(x,lines=True)
            activities = ['non_elimination', 'urination', 'defecation']
            for val in df_json['Tags'][0]:
                if val in activities:
                    tag=val
                    if tag=='non_elimination':
                        ne+=1
                        new_name =tag +str(ne)+ ".csv"
                        my_source =x.replace('json','csv')
                        my_dest =path +('/')+ new_name
                        os.rename(my_source, my_dest)

                        new_name_json = tag +str(ne)+ ".json"
                        my_source_json = x
                        print(my_source_json)
                        my_dest_json = path +('/')+ new_name_json
                        os.rename(my_source_json, my_dest_json)        


                    elif tag=='urination':
                        ur+=1
                        new_name =tag +str(ur)+ ".csv"
                        my_source =x.replace('json','csv')
                        my_dest =path +('/')+ new_name
                        os.rename(my_source, my_dest)

                        new_name_json = tag +str(ur)+ ".json"
                        my_source_json = x
                        print(my_source_json)
                        my_dest_json = path +('/')+ new_name_json
                        os.rename(my_source_json, my_dest_json)

                    else:
                        df+=1
                        new_name =tag +str(df)+ ".csv"
                        my_source =x.replace('json','csv')
                        my_dest =path +('/')+ new_name
                        os.rename(my_source, my_dest)

                        new_name_json = tag +str(df)+ ".json"
                        my_source_json = x
                        print(my_source_json)
                        my_dest_json = path +('/')+ new_name_json
                        os.rename(my_source_json, my_dest_json)                  


                else:
                    continue

            print(i)
            i+=1

        else:
            i+=1
            continue

## 2. Creation of new csv files with outlier Treatment and all features

In [ ]:
import os
rootdir = "D:/Ajit/pip/sample_labeled/"
folder_list = []
for subdir, dirs, files in os.walk(rootdir):
    folder_list.append(subdir)
del folder_list[0]

### i. Creating empty folder with folder numbers:

In [ ]:
for path in folder_list:
    path=path.replace('sample_labeled','sample_labeled8')
    os.mkdir(path)

### ii. Outlier treatment with flooring & capping:

In [ ]:
def outlier_capping(col_name):
    Q1 = 0
    Q3 = df[col_name].quantile(0.98)
    IQR = Q3
     
    upper_bound = Q3 + 3.5 * IQR   # Whisker width is 4
    no_upper_outlier=len(df[df[col_name]>upper_bound])
    print("number of outlier: "+str(no_upper_outlier))
    df[col_name]=np.where(df[col_name]>upper_bound,upper_bound,df[col_name])

### iii. creating new csv files in created folders with outlier treatment and new features:
#### 1.Differential load
#### 2. Sum of absolute value of differential load
#### 3. Acceleration
#### 4. Velocity
#### 5. Energy
#### 6. Power
#### 7. Momentum

In [ ]:
for folder in folder_list:
    path=folder

    for filename in os.listdir(path):
        x=path+'/'+filename
        if filename.endswith(".csv"):
            print(x)
            df=pd.read_csv(x)
            df_json=pd.read_json(x.replace('csv','json'),lines=True)

            df['name'] = df_json['Name_of_cat'][0]
            df['weight'] = df_json['Weight_of_cat'][0]
            activities = ['non_elimination', 'urination', 'defecation']
            tag=""
            for val in df_json['Tags'][0]:
                if val in activities:
                    tag=val

            df['activity']=tag

# difference of load from it's previous load (incremental or decremental load) and converted to abslolute value
            df['diff_lc0']= df['lc0'].diff().abs()
            df['diff_lc1']= df['lc1'].diff().abs()
            df['diff_lc2']= df['lc2'].diff().abs()
            df['diff_lc3']= df['lc3'].diff().abs()
            
# sum of all the loads as in aggregated datasheet all the load sensor's are corelated so we will work with only sum of all load sensor's differential value
            df['diff_load_sum']= df['diff_lc0']+df['diff_lc1']+df['diff_lc2']+df['diff_lc3']
            
# Outlier treatment with Q1=0 (because all differential valus are converted to absolute), Q3=0.98 quantile and whisker width = 3.5 and 
            outlier_capping('diff_load_sum')

# w = Weight of cat 
            w = df_json['Weight_of_cat'][0]
    
## Force = Mass * Acceleration
## Acceleration (a) = Force / Weight of cat (w)
            df['acc']= df['diff_load_sum'].apply(lambda x: x/w)

## Velocity, v2 = (a1*t) + v1
            df['velo'] = 0
            for i in range (1,len(df)):
                df['velo'][i]=(df['acc'][i]*0.025)+(df['velo'][i-1])

## Energy (Kinetic Energy), e = 0.5(w*v**2) 
            df['enr']= df['velo'].apply(lambda x: 0.5*w*(x**2))

## Power = Force * Velocity
            df['power']= df['velo']*df['diff_load_sum']

## Momentum = Mass * Velocity
            df['mom']=w*df['velo']

            filename = x
            filename=filename.replace('sample_labeled','sample_labeled8')

            df.to_csv(filename)

## 3. Creation of Final csv file:
#### Following are the columns:
#### 1. Name
#### 2. Activity
#### 3. Weight
#### 4. Duration 
#### 5. Event time
#### 6. Maximum value of "sum of differential load"
#### 7. mean of "sum of differential load"
#### 8. mean of Varience
#### 9. Skewness
#### 10. Kurtosis
#### 11. 90th percentile
#### 12. Acceleration
#### 13. Velocity
#### 14. Energy
#### 15. Power
#### 16. Momentum

In [ ]:
df_main = pd.DataFrame(columns = ['name' ,'activity' ,'weight', 'duration','start_time' ,'mean_diff_sum', 'variance',  'maximim', 'median', 'skewness', 'kurtosis', '0.9_quantile', 'acceleration', 'velocity', 'energy', 'power', 'momentum'])
df_main

In [ ]:
def value_insertion(loc_csv):
    df=pd.read_csv(loc_csv)
    print(loc_csv)
    df_main.loc[len(df_main.index)] = [df['name'][0], df['activity'][0], df['weight'][0], df['timestamp'][len(df)-1], df['start_time'][0], df['diff_load_sum'].mean(), df['diff_load_sum'].var(), df['diff_load_sum'].max(), df['diff_load_sum'].median(), df['diff_load_sum'].skew(), df['diff_load_sum'].kurtosis(), df['diff_load_sum'].quantile(0.9), df['acc'].mean(), df['velo'].mean(), df['enr'].mean(), df['power'].mean(), df['mom'].mean()]
    return df_main

In [ ]:
rootdir = "D:/Ajit/pip/sample_labeled5"
folder_list = []
for subdir, dirs, files in os.walk(rootdir):
    folder_list.append(subdir)
del folder_list[0]

In [ ]:
i=0
for folder in folder_list:
    path=folder
    for filename in os.listdir(path):
        x=path+'/'+filename
        print(i)
        value_insertion(x)
        i+=1

## 4. Adding a column in final data sheet if required:

In [ ]:
rootdir = 'C:/Users/ajit.mishra/Downloads/practice/sample/'
folder_list = []
for subdir, dirs, files in os.walk(rootdir):
    folder_list.append(subdir)
del folder_list[0]

In [ ]:
new_name = input('Enter new column name: ')
used_name = input('Enter the name of used column: ')
df_main[new_name] = ""
i=0
for loc_folder in folder_list:
    os.chdir(loc_folder)
    for FileList in glob.glob('*.csv'):
        loc_csv=loc_folder+'/'+FileList
        print(loc_csv)
        df_csv=pd.read_csv(loc_csv)
        val = df_csv[used_name].max()
        print(val)
        df_main.loc[i,new_name]=val
        print(i)
        i+=1